# Bibliography Categorization: 'BibCat'
## Tutorial: Machine learning (ML) models in bibcat.



---


## Introduction.

In this tutorial, we will use bibcat to train a machine learning (ML) model on some raw input text.


---

## User Workflow: Training a machine learning (ML) model.


The `Operator` class contains a user-friendly method `train_model_ML` that runs the full workflow for training an ML model, from the input raw text all the way to saving the output ML model.  We overview how this method can be run in the code blocks below.

For this tutorial, we have two sets of data: either 1) some short, made-up text for a quick run of the code, or 2) an imported database of text from an external file of the user's choosing. The former case is useful for getting a quick sense of how the code works. The latter case is useful for building an actual model, but of course will take much longer on larger databases of text!

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Import external packages
import os
import time
import json

In [3]:
# Set up for fetching necessary bibcat modules for the tutorial
# Check work directories: src/ is where all source python scripts are available. 
current_dir= os.path.dirname(os.path.abspath('__file__'))
_parent = os.path.dirname(current_dir)
src_dir = os.path.join(_parent, "src")

print(f'Current Directory: {current_dir}')
print(f'Source directory: {src_dir}')

# move to the ../src/ directory to import necessary modules. 
os.chdir(src_dir)

Current Directory: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/docs
Source directory: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src


In [4]:
#Import bibcat packages
import bibcat_classes as bibcat
import bibcat_config as config
import bibcat_constants as preset
import bibcat_parameters as params #Temporary file until contents moved elsewhere

Root directory =/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src, parent directory=/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat
/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src/models folder already exists.
/Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/output folder already exists.


In [5]:
#Which data would you like to run the ML model on?  Choose from the booleans below.
do_quick_run = False #This will train the ML model on short bits of text. Runs pretty quickly.
do_real_run = True #This will train the ML model on external text. Will take longer for larger databases.
#

In [6]:
#The rest of these parameters can be left as-is for a first run-through.
#
num_papers = 500 #500 #None, or an integer; if an integer, will truncate external .json text dataset to this size
#Set num_papers=None to use all available papers in external dataset
#Note: If set to integer, final paper count might be a little more than target num_papers given
#
allowed_classifications = config.allowed_classifications #For external data; classifications to include
mapper = params.map_papertypes #For masking of classes (e.g., masking 'supermention' as 'mention')
#

#Fetch filepaths for model and data
name_model = config.name_model
filepath_json = config.path_json
dir_model = os.path.join(config.dir_allmodels, name_model)
#
#Set values for generating ML model
do_reuse_run = True #Whether or not to reuse any existing output from previous training+validation+testing runs
do_shuffle = True #Whether or not to shuffle contents of training vs validation vs testing datasets
fraction_TVT = [0.8, 0.1, 0.1] #Fractional breakdown of training vs validation vs testing datasets
#
mode_TVT = "uniform" # "uniform" #"available"
#"uniform" = all training datasets will have the same number of entries
#"available" = all training datasets will use full fraction (from fraction_TVT) of data available
#
seed_TVT = 10 #Random seed for generating training vs validation vs testing datasets
seed_ML = 8 #Random seed for ML model
mode_modif = "skim_trim_anon" #Mode to use for processing and generating modifs from input raw text
#NOTE: See other modif modes in workflow tutorial
buffer = 0
#
#Prepare some Keyword objects
#kobj_hubble = bibcat.Keyword(
#                keywords=["Hubble", "Hubble Telescope",
#                          "Hubble Space Telescope"],
#                acronyms=["hst", "ht"])
#all_kobjs = [kobj_hubble]
all_kobjs = params.all_kobjs
#

In [7]:
#Initialize an empty ML classifier
classifier_ML = bibcat.Classifier_ML(filepath_model=None, fileloc_ML=None,
                                    class_names=None, do_verbose=True)
#

In [8]:
#Initialize an Operator
tabby_ML = bibcat.Operator(classifier=classifier_ML, mode=mode_modif, keyword_objs=all_kobjs,
                           do_verbose=True, load_check_truematch=False, do_verbose_deep=False)
#

Instance of Operator successfully initialized!
Keyword objects:
0: Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['HST', 'HT']

1: Keyword Object:
Name: Webb Telescope
Keywords: ['James Webb Space Telescope', 'Webb Space Telescope', 'James Webb Telescope', 'Webb Telescope']
Acronyms: ['JWST', 'JST', 'JT']

2: Keyword Object:
Name: Transiting Exoplanet Survey Satellite
Keywords: ['Transiting Exoplanet Survey Satellite']
Acronyms: ['TESS']

3: Keyword Object:
Name: KEPLER
Keywords: ['KEPLER']
Acronyms: []

4: Keyword Object:
Name: Pan-STARRS
Keywords: ['Panoramic Survey Telescope and Rapid Response System', 'Pan-STARRS1', 'Pan-STARRS']
Acronyms: ['PanSTARRS1', 'PanSTARRS', 'PS1']

5: Keyword Object:
Name: Galaxy Evolution Explorer
Keywords: ['Galaxy Evolution Explorer']
Acronyms: ['GALEX']

6: Keyword Object:
Name: K2
Keywords: ['K2']
Acronyms: []

7: Keyword Object:
Name: Hubble Legacy Archive
Keywords: ['Hubble Legacy Archive'

In [9]:
#Set up data for the quick example case. But in reality, ML models should be trained on MUCH more data than this!!!
if do_quick_run:
    #Make some fake data
    dict_texts_raw = {"science":["We present HST observations in Figure 4.",
                            "The HST stars are listed in Table 3b.",
                            "Despite our efforts to smooth the data, there are still rings in the HST images.",
                            "See Section 8c for more discussion of the Hubble images.",
                            "The supernovae detected with HST tend to be brighter than initially predicted.",
                            "Our spectra from HST fit well to the standard trend first published in Someone et al. 1990.",
                            "We use the Hubble Space Telescope to build an ultraviolet database of the target stars.",
                            "The blue points (HST) exhibit more scatter than the red points (JWST).",
                            "The benefit, then, is the far higher S/N we achieved in our HST observations.",
                            "Here we employ the Hubble Telescope to observe the edge of the photon-dominated region.",
                            "The black line shows that the region targeted with Hubble has an extreme UV signature."],
                     "datainfluenced":["The simulated Hubble data is plotted in Figure 4.",
                           "Compared to the HST observations in Someone et al., our JWST follow-up reached higher S/N.",
                           "We were able to reproduce the luminosities from Hubble using our latest models.",
                           "We overplot Hubble-observed stars from Someone et al. in Figure 3b.",
                           "We built the spectral templates using UV data in the Hubble archive.",
                           "We simulate what our future HST observations will look like to predict the S/N.",
                           "Our work here with JWST is inspired by our earlier HST study published in 2010.",
                           "We therefore use the Hubble statistics from Author et al. to guide our stellar predictions.",
                           "The stars in Figure 3 were plotted based on the HST-fitted trend line in Person et al.",
                           "The final step is to use the HST exposure tool to put our modeled images in context."],
                     "mention":["Person et al. used HST to measure the Hubble constant.",
                            "We will present new HST observations in a future work.",
                            "HST is do_a fantastic instrument that has revolutionized our view of space.",
                            "The Hubble Space Telescope (HST) has its mission center at the STScI.",
                            "We can use HST to power a variety of science in the ultraviolet regime.",
                            "It is not clear when the star will be observable with HST.",
                            "More data can be found and downloaded from the Hubble archive.",
                            "We note that HST can be used to observe the stars as well, at higher S/N.",
                            "However, we ended up using the JWST rather than HST observations in this work.",
                            "We push the analysis of the Hubble component of the dataset to a future study.",
                            "We expect the HST observations to be released in the fall.",
                            "We look forward to any follow-up studies with, e.g., the Hubble Telescope."]}
    #
    #Convert into dictionary with: key:text,class,id,mission structure
    i_track = 0
    dict_texts = {}
    for key in dict_texts_raw:
        curr_set = dict_texts_raw[key]
        for ii in range(0, len(curr_set)):
            dict_texts[str(i_track)] = {"text":curr_set[ii], "class":key, "id":"{0}_{1}".format(key, ii),
                                       "mission":"HST", "bibcode":"{0}_{1}".format(key, ii)}
            i_track += 1
#

In [10]:
#Set up data for the external data case.
if do_real_run:
    #Load the original data
    with open(filepath_json, 'r') as openfile:
        dataset = json.load(openfile)
        len(dataset)
    #
    #Initialize holder to keep track of bibcodes used (avoids duplicate dataset entries)
    list_bibcodes = []
    #
    #Organize a new version of the data with: key:text,class,id,mission structure
    i_track = 0 #Track number of papers kept from original dataset
    dict_texts = {}
    for ii in range(0, len(dataset)):
        #Extract mission classifications for current text
        curr_data = dataset[ii]
        #
        #Skip if no valid text at all for this text
        if ("body" not in curr_data):
            continue
        #
        #Skip if no valid missions at all for this text
        if ("class_missions" not in curr_data):
            continue
        #
        #Otherwise, extract the bibcodes and missions
        curr_bibcode = curr_data["bibcode"]
        curr_missions = curr_data["class_missions"]
        print(curr_bibcode)
        print(curr_missions)
        #
        #Skip if bibcode already encountered (and so duplicate entry)
        if (curr_bibcode in list_bibcodes):
            print("Duplicate bibcode encountered: {0}. Skipping.".format(curr_bibcode))
            continue
        #
        #Iterate through missions for this text
        i_mission = 0
        for curr_key in curr_missions:
            #If this is not an allowed mission, skip
            if (curr_missions[curr_key]["papertype"] not in allowed_classifications):
                continue
            #
            #Otherwise, check if this mission is a target mission
            fetched_kobj = tabby_ML._fetch_keyword_object(lookup=curr_key,
                                                          do_verbose=False, do_raise_emptyerror=False)
            #Skip if not a target
            if (fetched_kobj is None):
                continue
            #
            #Otherwise, store classification info for this entry
            curr_class = curr_missions[curr_key]["papertype"]
            new_dict = {"text":curr_data["body"], #Text for this paper
                        "bibcode":curr_data["bibcode"], #Bibcode for this paper
                        "class":curr_class, #Classification for this mission
                        "mission":curr_key, #The mission itself
                        "id":("paper{0}_mission{1}_{2}_{3}".format(ii, i_mission,
                                                                   curr_key, curr_class)) #ID for this entry
                       }
            dict_texts[str(i_track)] = new_dict
            #
            #Increment counters
            i_mission += 1 #Count of kept missions for this paper
            i_track += 1 #Count of kept classifications overall
        #
        
        #Record this bibcode as stored
        list_bibcodes.append(curr_bibcode)

        #Terminate early if requested number of papers reached
        if ((num_papers is not None) and (i_track >= num_papers)):
            break
    #
#

19782

2021MNRAS.500.3002B
{}
2021MNRAS.500.3254R
{'Roman': {'bibcode': '2021MNRAS.500.3254R', 'papertype': 'MENTION'}, 'PanSTARRS': {'bibcode': '2021MNRAS.500.3254R', 'papertype': 'DATA_INFLUENCED'}}
2020SSRv..216...50C
{'JWST': {'bibcode': '2020SSRv..216...50C', 'papertype': 'MENTION'}}
2020SSRv..216...69A
{'HST': {'bibcode': '2020SSRv..216...69A', 'papertype': 'DATA_INFLUENCED'}, 'JWST': {'bibcode': '2020SSRv..216...69A', 'papertype': 'MENTION'}, 'Roman': {'bibcode': '2020SSRv..216...69A', 'papertype': 'MENTION'}}
2020SSRv..216...38H
{}
2020SSRv..216...81A
{}
2021MNRAS.500.4004D
{'PanSTARRS': {'bibcode': '2021MNRAS.500.4004D', 'papertype': 'MENTION'}}
2020SSRv..216..124V
{'GALEX': {'bibcode': '2020SSRv..216..124V', 'papertype': 'MENTION'}, 'PanSTARRS': {'bibcode': '2020SSRv..216..124V', 'papertype': 'MENTION'}, 'HST': {'bibcode': '2020SSRv..216..124V', 'papertype': 'MENTION'}}
2020SSRv..216...29P
{'JWST': {'bibcode': '2020SSRv..216...29P', 'papertype': 'MENTION'}}
2021MNRAS.500.5229E
{'Pan

2020SPIE11463E..20T
{}
2020SPIE11354E..0AL
{}
2020SPIE11297E..05O
{}
2020SPIE11413E..0ZT
{}
2020SPIE11522E..03P
{}
2020SPIE11369E..1LP
{}
2020SPIE11279E..1UL
{}
2020SPIE11345E..07Z
{}
2020SPIE11302E..1WP
{}
2020SPIE11345E..1SK
{}
2020SPIE11465E..1FM
{}
2020SPIE11277E..0NH
{}
2020SPIE11402E..0BS
{}
2020SPIE11388E..0PB
{}
2020SPIE11358E..20K
{}
2020SPIE11455E..6GW
{}
2020SPIE11369E..1FK
{}
2020SPIE11382E..0CC
{}
2020SPIE11312E..1QS
{}
2020SPIE11386E..0OA
{}
2020SPIE11442E..0CM
{}
2020SPIE11419E..0BN
{}
2020SPIE11375E..07A
{'HST': {'bibcode': '2020SPIE11375E..07A', 'papertype': 'MENTION'}}
2020SPIE11479E..0JS
{'HST': {'bibcode': '2020SPIE11479E..0JS', 'papertype': 'MENTION'}}
2020SPIE11290E..1BL
{}
2020SPIE11457E..0OL
{}
2020SPIE11522E..10B
{}
2020SPIE11369E..16O
{}
2020SPIE11296E..2SS
{}
2020SPIE11355E..0OB
{}
2020SPIE11422E..09L
{}
2020SPIE11281E..1QD
{}
2020SPIE11331E..0XR
{}
2020SPIE11524E..26K
{}
2020SPIE11524E..0OH
{}
2020SPIE11433E..0NB
{}
2020SPIE11479E..0GR
{'JWST': {'bibcode': '

2020PhRvD.102f3504K
{'Roman': {'bibcode': '2020PhRvD.102f3504K', 'papertype': 'MENTION'}}
2020PhRvD.101b4013B
{}
2020PhRvL.124t0502W
{}
2020PhRvD.102d3516P
{}
2020RAA....20..157W
{'KEPLER': {'bibcode': '2020RAA....20..157W', 'papertype': 'MENTION'}}
2020PhRvD.101j5019I
{}
2020PhRvD.102b3026M
{'Roman': {'bibcode': '2020PhRvD.102b3026M', 'papertype': 'MENTION'}}
2020PhRvD.102f3006S
{}
2020Sci...369.1597R
{}
2020PhRvD.102h3027O
{}
2020PhRvL.124s2001I
{}
2020PhRvD.101h3031D
{}
2020PhRvD.102b3005D
{}
2020PhRvD.101l3014C
{}
2020PhRvD.101f3019W
{}
2020PhRvL.124t6402P
{}
2020PhRvL.124u7701F
{}
2020PhRvD.101a6002L
{}
2020PhRvD.101c4026G
{}
2020PhRvD.101d3021F
{}
2020PhRvD.101b1501B
{}
2020PhRvD.102h4056M
{'JWST': {'bibcode': '2020PhRvD.102h4056M', 'papertype': 'MENTION'}, 'Roman': {'bibcode': '2020PhRvD.102h4056M', 'papertype': 'MENTION'}}
2020PhRvL.125w7004L
{}
2020PhRvL.124b5001M
{}
2020PhRvD.101e6006S
{}
2020PhRvL.124t0503M
{}
2020PhRvD.102j3520L
{}
2020PhRvD.101l3503M
{}
2020PhRvD.102h4016R

{'HST': {'bibcode': '2020RAA....20...15A', 'papertype': 'SCIENCE'}}
2020PASP..132b4503C
{'HST': {'bibcode': '2020PASP..132b4503C', 'papertype': 'MENTION'}}
2020P&SS..18304627G
{'HST': {'bibcode': '2020P&SS..18304627G', 'papertype': 'SCIENCE'}}
2020PhRvD.101b8501G
{}
2020PhRvD.101f4006C
{}
2020PASP..132j5001H
{'TESS': {'bibcode': '2020PASP..132j5001H', 'papertype': 'MENTION'}}
2020PhRvD.102h4022J
{}
2020PSJ.....1...60D
{'HST': {'bibcode': '2020PSJ.....1...60D', 'papertype': 'MENTION'}}
2020PhRvD.102h3005W
{}
2020PhRvD.102l1701C
{}
2020PASA...37...25N
{'GALEX': {'bibcode': '2020PASA...37...25N', 'papertype': 'MENTION'}, 'JWST': {'bibcode': '2020PASA...37...25N', 'papertype': 'MENTION'}, 'Roman': {'bibcode': '2020PASA...37...25N', 'papertype': 'MENTION'}}
2020PhRvD.102l4040G
{}
2020PhRvD.101b3002F
{}
2020PASP..132l5001Z
{'HST': {'bibcode': '2020PASP..132l5001Z', 'papertype': 'MENTION'}, 'PanSTARRS': {'bibcode': '2020PASP..132l5001Z', 'papertype': 'DATA_INFLUENCED'}}
2020PASJ...72...24L
{}

In [11]:
#Throw error if not enough text entries collected
if do_real_run:
    if ((num_papers is not None) and (len(dict_texts) < num_papers)):
        raise ValueError("Err: Something went wrong during initial processing. Insufficient number of texts extracted."
                        +"\nRequested number of texts: {0}\nActual number of texts: {1}"
                        .format(num_papers, len(dict_texts)))
#

In [12]:
#Uncomment the code below to print a snippet of each of the entries in the dataset.
#"""
print("Number of processed texts: {0}={1}\n".format(i_track, len(dict_texts)))
for curr_key in dict_texts:
    print("Text #{0}:".format(curr_key))
    print("Classification: {0}".format(dict_texts[curr_key]["class"]))
    print("Mission: {0}".format(dict_texts[curr_key]["mission"]))
    print("ID: {0}".format(dict_texts[curr_key]["id"]))
    print("Bibcode: {0}".format(dict_texts[curr_key]["bibcode"]))
    print("Text snippet:")
    print(dict_texts[curr_key]["text"][0:500])
    print("---\n\n")
#"""

Number of processed texts: 500=500

Text #0:
Classification: DATA_INFLUENCED
Mission: PanSTARRS
ID: paper1_mission0_PanSTARRS_DATA_INFLUENCED
Bibcode: 2021MNRAS.500.3254R
Text snippet:
1 INTRODUCTION The acceleration of the expansion of the Universe discovered about 20 yr ago (Riess et al. 1998 ; Perlmutter et al. 1999 ) set a key milestone in cosmology history: current observations can be accounted for with the ΛCDM standard model, but at the cost of introducing a dark energy component, making up today ∼70 per cent of the energy content of the Universe. Around the same time, the SDSS collaboration (York et al. 2000 ) initiated spectroscopic observations to study large-scale s
---


Text #1:
Classification: MENTION
Mission: JWST
ID: paper2_mission0_JWST_MENTION
Bibcode: 2020SSRv..216...50C
Text snippet:
Space Sci Rev (2020) 216:50 https://doi.org/10.1007/s11214-020-00674-x The Molecular Cloud Lifecycle Mélanie Chevance1 · J.M. Diederik Kruijssen1 · Enrique Vazquez-Semadeni2 · Fumitaka 

Text #126:
Classification: MENTION
Mission: K2
ID: paper108_mission1_K2_MENTION
Bibcode: 2021AJ....161...19G
Text snippet:
1. Introduction High-precision photometry with NASA’s Kepler space mission revealed the existence of a large population of transiting planets with radii between those of Earth and Neptune and with periods shorter than 100 days (e.g., Borucki et al. 2011 ; Howard et al. 2012 ; Batalha et al. 2013 ; Dressing Charbonneau 2013 ; Fressin et al. 2013 ; Petigura et al. 2013 ). Thanks to more precise measurements of the stellar radii of the Kepler field, first via spectroscopy (Petigura et al. 2017 ; Fu
---


Text #127:
Classification: SCIENCE
Mission: HST
ID: paper108_mission2_HST_SCIENCE
Bibcode: 2021AJ....161...19G
Text snippet:
1. Introduction High-precision photometry with NASA’s Kepler space mission revealed the existence of a large population of transiting planets with radii between those of Earth and Neptune and with periods shorter than 100 days (e.g., Borucki et 

ID: paper1089_mission0_HST_DATA_INFLUENCED
Bibcode: 2020PhRvD.102f3530W
Text snippet:
I. INTRODUCTION With more extensive surveys at different scales and improved measuring techniques, measurements of late-time cosmic acceleration and growth of gravitational structure have sharpened considerably in recent years [1] . Independent observations from Planck-2018 cosmic microwave background (CMB) radiation have been tighter than before [2–4] . Type Ia supernovae (SNe Ia) [5,6] and baryon acoustic oscillations (BAO) [7,8] have been measured up to redshift , and we now have obtained dat
---


Text #314:
Classification: DATA_INFLUENCED
Mission: HST
ID: paper1099_mission0_HST_DATA_INFLUENCED
Bibcode: 2020PhRvD.102d3024H
Text snippet:
I. INTRODUCTION The standard cosmological model incorporates dark matter (DM) in the simplest way possible, i.e., a noninteracting cold matter component with constant equation of state throughout its cosmological evolution. A scenario of this type is not only simpl

In [13]:
#Print number of texts that fell under given parameters
print("Target missions:")
for curr_kobj in all_kobjs:
    print(curr_kobj)
    print("")
print("")
print("Number of valid text entries:")
print(len(dict_texts))

Target missions:
Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['HST', 'HT']


Keyword Object:
Name: Webb Telescope
Keywords: ['James Webb Space Telescope', 'Webb Space Telescope', 'James Webb Telescope', 'Webb Telescope']
Acronyms: ['JWST', 'JST', 'JT']


Keyword Object:
Name: Transiting Exoplanet Survey Satellite
Keywords: ['Transiting Exoplanet Survey Satellite']
Acronyms: ['TESS']


Keyword Object:
Name: KEPLER
Keywords: ['KEPLER']
Acronyms: []


Keyword Object:
Name: Pan-STARRS
Keywords: ['Panoramic Survey Telescope and Rapid Response System', 'Pan-STARRS1', 'Pan-STARRS']
Acronyms: ['PanSTARRS1', 'PanSTARRS', 'PS1']


Keyword Object:
Name: Galaxy Evolution Explorer
Keywords: ['Galaxy Evolution Explorer']
Acronyms: ['GALEX']


Keyword Object:
Name: K2
Keywords: ['K2']
Acronyms: []


Keyword Object:
Name: Hubble Legacy Archive
Keywords: ['Hubble Legacy Archive']
Acronyms: ['HLA']



Number of valid text entries:
500


In [14]:
#Use the Operator instance to train an ML model
start=time.time()
tabby_ML.train_model_ML(dir_model=dir_model, name_model=name_model, do_reuse_run=do_reuse_run,
                        seed_ML=seed_ML, seed_TVT=seed_TVT, dict_texts=dict_texts, mapper=mapper,
                        buffer=buffer, fraction_TVT=fraction_TVT, mode_TVT=mode_TVT, do_shuffle=do_shuffle,
                        do_verbose=True, do_verbose_deep=False)

print(f'Time to train the model with run = {time.time()-start} seconds.')
#


> Running train_model_ML()!
Processing text data into modifs...
25 of 500 total texts have been processed...
50 of 500 total texts have been processed...
75 of 500 total texts have been processed...
100 of 500 total texts have been processed...
125 of 500 total texts have been processed...
150 of 500 total texts have been processed...
175 of 500 total texts have been processed...
200 of 500 total texts have been processed...
225 of 500 total texts have been processed...
250 of 500 total texts have been processed...
275 of 500 total texts have been processed...
300 of 500 total texts have been processed...
325 of 500 total texts have been processed...
350 of 500 total texts have been processed...
375 of 500 total texts have been processed...
400 of 500 total texts have been processed...
425 of 500 total texts have been processed...
450 of 500 total texts have been processed...
475 of 500 total texts have been processed...
500 of 500 total texts have been processed...
500 texts have bee

INFO:absl:Using /var/folders/0g/b03r16sd1kl3_j3fml_xwl9h0002td/T/tfhub_modules to cache modules.


Found 173 files belonging to 3 classes.
Loading validation data...
Found 22 files belonging to 3 classes.
Loading testing data...
Found 305 files belonging to 3 classes.
Done loading datasets.
Loading ML model components...


INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'.


Loaded ML preprocessor: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


...

INFO:absl:using Lamb optimizer


Done building an empty ML model.
Setting up loss, metric, and optimization functions...
# of training steps: 30
# of warmup steps: 3
Type of optimizer and initial lr: lamb, 3e-05
Done compiling loss, metric, and optimization functions.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessor (KerasLayer)      {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),            

INFO:tensorflow:Assets written to: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src/models/my_test_run/tfoutput_my_test_run/assets


INFO:tensorflow:Assets written to: /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src/models/my_test_run/tfoutput_my_test_run/assets



Training complete.
New ML model trained and stored in /Users/jamila.pegues/Documents/STScI_Fellowship/Functional/Library/BibTracking/repo_stsci/bibcat/src/models/my_test_run.
Run of train_model_ML() complete!

Time to train the model with run = 625.5580139160156 seconds.


And with that, we're done training a new ML model!  If run successfully, the model will be saved in the `dir_model` directory.

We can then use the brand new model to classify some new text, like so:

In [15]:
#Set path to the new model output
filepath_model = os.path.join(dir_model, (name_model+".npy"))
fileloc_ML = os.path.join(dir_model, (preset.tfoutput_prefix+name_model))
#Load the new ML model into a new Classifier_ML instance
classifier_ML = bibcat.Classifier_ML(filepath_model=filepath_model, fileloc_ML=fileloc_ML,
                                    do_verbose=True)
#
#Load the instance into a new Operator
tabby_ML = bibcat.Operator(classifier=classifier_ML, mode=mode_modif, keyword_objs=all_kobjs,
                           do_verbose=True, load_check_truematch=True, do_verbose_deep=False)
#

INFO:absl:using Lamb optimizer


Instance of Operator successfully initialized!
Keyword objects:
0: Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['HST', 'HT']

1: Keyword Object:
Name: Webb Telescope
Keywords: ['James Webb Space Telescope', 'Webb Space Telescope', 'James Webb Telescope', 'Webb Telescope']
Acronyms: ['JWST', 'JST', 'JT']

2: Keyword Object:
Name: Transiting Exoplanet Survey Satellite
Keywords: ['Transiting Exoplanet Survey Satellite']
Acronyms: ['TESS']

3: Keyword Object:
Name: KEPLER
Keywords: ['KEPLER']
Acronyms: []

4: Keyword Object:
Name: Pan-STARRS
Keywords: ['Panoramic Survey Telescope and Rapid Response System', 'Pan-STARRS1', 'Pan-STARRS']
Acronyms: ['PanSTARRS1', 'PanSTARRS', 'PS1']

5: Keyword Object:
Name: Galaxy Evolution Explorer
Keywords: ['Galaxy Evolution Explorer']
Acronyms: ['GALEX']

6: Keyword Object:
Name: K2
Keywords: ['K2']
Acronyms: []

7: Keyword Object:
Name: Hubble Legacy Archive
Keywords: ['Hubble Legacy Archive'

In [16]:
#Run the classifier for some sample text below
lookup = "HST"
text = "In this study, we present our lovely HST observations of bright stars in the nearby star-forming region Taurus."
threshold = 0.8
#
#Run the classifier
result = tabby_ML.classify(text=text, lookup=lookup, buffer=0, threshold=threshold,
                            do_raise_innererror=False, do_check_truematch=True)
#

> Running _fetch_keyword_object() for lookup term HST.
Best matching keyword object (keyobj) for keyword HST:
Keyword Object:
Name: Hubble
Keywords: ['Hubble Space Telescope', 'Hubble Telescope', 'Hubble']
Acronyms: ['HST', 'HT']


Preprocessing and extracting modifs from the text...

Running Grammar on the text...
Text has been processed into modifs.
Text has been processed into modif.

Running classify_text for ML classifier:
Class names from model:
['data_influenced', 'mention', 'science']


Method classify_text for ML classifier complete!
Max verdict: data_influenced



In [17]:
#Print the classifier results
print("Modif: {2}\n\nClassification: {0}\n\nUncertainties per class: {1}\n"
      .format(result["verdict"], result["uncertainty"], result["modif"]))
print("Full classification output:\n{0}".format(result))

Modif: In this study, we present our OBJ observations of stars in the star - forming region Taurus.

Classification: z_lowprob

Uncertainties per class: {'data_influenced': 0.38300177, 'mention': 0.33406982, 'science': 0.28292844}

Full classification output:
{'verdict': 'z_lowprob', 'scores_comb': None, 'scores_indiv': None, 'uncertainty': {'data_influenced': 0.38300177, 'mention': 0.33406982, 'science': 0.28292844}, 'modif': 'In this study, we present our OBJ observations of stars in the star - forming region Taurus.'}


---

In [18]:
#Set end marker for this tutorial.
print("This tutorial completed successfully.")

This tutorial completed successfully.
